In [3]:
#26/8/24 Creates a K fold RF model for padel data, a copy of the fingerprint one

import os
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.use_inf_as_na = True

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Padel/3D/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    df = df.drop(['Name'], axis=1)
    df = df.to_numpy()
    df = np.nan_to_num(df)
    df = pd.DataFrame(df)
    df = df.rename(columns={0: "Toxicity_Value"})
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 50):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    sets += 1

/var/folders/cq/5cl8ry195dj0219k6rt6csbm0000gn/T/ipykernel_27018/850714280.py:16: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


For dataset Hepatotoxicity_PaDEL.csv
for fold 1 test set mcc of 0.49600124840483517 valid set mcc of 0.42052376676874836
for fold 2 test set mcc of 0.4750414349009691 valid set mcc of 0.37726102702676406
for fold 3 test set mcc of 0.3976510924728902 valid set mcc of 0.5943138365192613
for fold 4 test set mcc of 0.4023837433448534 valid set mcc of 0.4880924080189066
for fold 5 test set mcc of 0.4433523596321591 valid set mcc of 0.4872010833210581

validation metrics of:
positives in data 293
negatives in data 283
net accuracy = 0.7482638888888888
mcc = 0.4963453421843115
For dataset NR-PPAR-gamma_PaDEL.csv
for fold 1 test set mcc of 0.6057576709293282 valid set mcc of 0
for fold 2 test set mcc of 0.33008622524996895 valid set mcc of 0
for fold 3 test set mcc of 0.7238095238095238 valid set mcc of 0.4385566936919029
for fold 4 test set mcc of 0.6589657274544057 valid set mcc of 0.37446543094704876
for fold 5 test set mcc of 0.5855676421026854 valid set mcc of 0.18638210221950421

validat

for fold 1 test set mcc of 0.4402043892400666 valid set mcc of 0.2193819762367038
for fold 2 test set mcc of 0.32050228477919135 valid set mcc of 0.2717914161090704
for fold 3 test set mcc of 0.601965601965602 valid set mcc of 0.2892346639593178
for fold 4 test set mcc of 0.43268345558071136 valid set mcc of 0.43465655192911734
for fold 5 test set mcc of 0.32632901395328684 valid set mcc of 0.6410336722814981

validation metrics of:
positives in data 101
negatives in data 103
net accuracy = 0.6715686274509803
mcc = 0.352291879832765
For dataset NR-ER_PaDEL.csv
for fold 1 test set mcc of 0.4091857683612964 valid set mcc of 0.3847937471446751
for fold 2 test set mcc of 0.3222934146943425 valid set mcc of 0.4002715433620869
for fold 3 test set mcc of 0.37344642773735687 valid set mcc of 0.3441294050765354
for fold 4 test set mcc of 0.4999370278772884 valid set mcc of 0.38467746775338796
for fold 5 test set mcc of 0.4992999722118545 valid set mcc of 0.2693263984957048

validation metrics o

In [5]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/PaDEL/data/3D/padel_rf.csv')

In [1]:
len(namelist)

NameError: name 'namelist' is not defined